# 第2回講義 宿題

## 課題
今Lessonで学んだことを元に、MNISTのファッション版 (Fashion MNIST、クラス数10) をソフトマックス回帰によって分類してみましょう。

Fashion MNISTの詳細については以下のリンクを参考にしてください。

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 90%

### ルール
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください。**
- **ソフトマックス回帰のアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)。
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません。

### 提出方法
- 2つのファイルを提出していただきます。
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し、**Homeworkタブから`chap02`を選択して**提出してください。
    2. それに対応するpythonのコードを`submission_code.py`として保存し、**Homeworkタブから`chap02 (code)`を選択して**提出してください。
      - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください（writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください）。
      
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します。
- 毎日夜24時にテストデータの一部に対する精度でLeader Boardを更新します。
- 締切日の夜24時にテストデータ全体に対する精度でLeader Boardを更新します。これを最終的な評価とします。

### データの読み込み (このセルは修正しないでください)

In [1]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('/root/userspace/public/chap02/data/x_train.npy')
    y_train = np.load('/root/userspace/public/chap02/data/y_train.npy')
    
    # テストデータ
    x_test = np.load('/root/userspace/public/chap02/data/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

# Verify data sets

In [2]:
x_train, y_train, x_test = load_fashionmnist()

In [3]:
x_train.shape

(60000, 784)

In [4]:
y_train.shape

(60000, 10)

In [5]:
y_train # one hot encoding

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
x_test.shape

(10000, 784)

In [7]:
x_train.shape[0]

60000

### ソフトマックス回帰の実装

In [8]:
#%%writefile /root/userspace/submission_code.py
# delete # above when you create .py file of this cell and submit itself from the submission form

x_train, y_train, x_test = load_fashionmnist()

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


# logの中身が0になるのを防ぐ
def np_log(x):
    return np.log(np.clip(a=x, a_min=1e-10, a_max=x))


def softmax(x):
    # WRITE ME
    x -= x.max(axis=1, keepdims=True)
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)
    

# weights
W = np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float') # WRITE ME
b = np.zeros(shape=(10,)).astype('float32')# WRITE ME

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

def train(x, t, eps=0.00005):
    # WRITE ME
    """
    :param x: np.ndarray, 入力データ, shape=(batch_size, 入力の次元数)
    :param t: np.ndarray, 教師ラベル, shape=(batch_size, 出力の次元数)
    :param eps: float, 学習率
    """
    global W, b
    
    
    batch_size = x.shape[0]
    
    
    # forward
    y = softmax(np.matmul(x, W) + b) # shape: (batch_size, 出力の次元数)
    
    # backward
    cost = (- t * np_log(y)).sum(axis=1).mean()
    delta = y - t # shape: (batch_size, 出力の次元数)
    
    # パラメータの更新
    dW = np.matmul(x.T, delta) / batch_size # shape: (入力の次元数, 出力の次元数)
    db = np.matmul(np.ones(shape=(batch_size,)), delta) / batch_size # shape: (出力の次元数,)
    W -= eps * dW
    b -= eps * db

    return cost

def valid(x, t):
    # WRITE ME
    y = softmax(np.matmul(x, W) + b)
    cost = (- t * np_log(y)).sum(axis=1).mean()
    
    return cost, y

for epoch in range(1000):
    # オンライン学習
    # WRITE ME
    x_train, y_train = shuffle(x_train, y_train)
    for x, t in zip(x_train, y_train):
        cost = train(x[None, :], t[None, :])
    cost, y_pred = valid(x_valid, y_valid)
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch + 1,
        cost,
        accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))

y_pred = softmax(np.matmul(x_test, W) + b) # WRITE ME    
y_pred = y_pred.argmax(axis=1)


submission = pd.Series(y_pred, name='label')
submission.to_csv('/root/userspace/submission_pred.csv', header=True, index_label='id')

EPOCH: 1, Valid Cost: 0.987, Valid Accuracy: 0.699
EPOCH: 2, Valid Cost: 0.831, Valid Accuracy: 0.736
EPOCH: 3, Valid Cost: 0.759, Valid Accuracy: 0.761
EPOCH: 4, Valid Cost: 0.713, Valid Accuracy: 0.774
EPOCH: 5, Valid Cost: 0.682, Valid Accuracy: 0.779
EPOCH: 6, Valid Cost: 0.658, Valid Accuracy: 0.787
EPOCH: 7, Valid Cost: 0.639, Valid Accuracy: 0.791
EPOCH: 8, Valid Cost: 0.625, Valid Accuracy: 0.796
EPOCH: 9, Valid Cost: 0.612, Valid Accuracy: 0.799
EPOCH: 10, Valid Cost: 0.601, Valid Accuracy: 0.801
EPOCH: 11, Valid Cost: 0.591, Valid Accuracy: 0.804
EPOCH: 12, Valid Cost: 0.583, Valid Accuracy: 0.808
EPOCH: 13, Valid Cost: 0.575, Valid Accuracy: 0.809
EPOCH: 14, Valid Cost: 0.569, Valid Accuracy: 0.810
EPOCH: 15, Valid Cost: 0.563, Valid Accuracy: 0.813
EPOCH: 16, Valid Cost: 0.558, Valid Accuracy: 0.815
EPOCH: 17, Valid Cost: 0.553, Valid Accuracy: 0.815
EPOCH: 18, Valid Cost: 0.549, Valid Accuracy: 0.818
EPOCH: 19, Valid Cost: 0.544, Valid Accuracy: 0.817
EPOCH: 20, Valid Cost

In [ ]:
'''
y_pred = softmax(np.matmul(x_test, W) + b) # WRITE ME
for i in range(y_pred.shape[0]):
    myindex = np.argmax(y_pred[i])
    y_pred[i][myindex] = 1
    y_pred[i][np.where(y_pred[i] != 1)] = 0
   
y_pred = [np.where(r==1)[0][0] for r in y_pred]
'''


In [24]:
y_pred.argmax(axis=1)

array([7, 8, 3, ..., 3, 9, 8])

In [27]:
x_train.shape[0]

54000

In [39]:
y_pred[0, :]

array([9.99978236e-01, 1.43005383e-23, 2.25108247e-14, 9.61981321e-18,
       9.17136694e-27, 2.24318352e-78, 2.17638157e-05, 1.23113529e-68,
       1.45486580e-26, 4.10439230e-68])

In [42]:
y_pred.shape[0]

10000

In [49]:
y_pred[0][0]

0.9999782361842975

In [52]:
y_pred[0][np.where(y_pred[0] < 0.9)]

array([1.43005383e-23, 2.25108247e-14, 9.61981321e-18, 9.17136694e-27,
       2.24318352e-78, 2.17638157e-05, 1.23113529e-68, 1.45486580e-26,
       4.10439230e-68])

In [62]:
y_pred

[0,
 6,
 9,
 9,
 6,
 5,
 7,
 2,
 3,
 8,
 6,
 5,
 6,
 9,
 9,
 7,
 1,
 3,
 9,
 7,
 1,
 9,
 5,
 9,
 6,
 6,
 3,
 8,
 6,
 5,
 4,
 0,
 6,
 6,
 3,
 8,
 6,
 8,
 4,
 5,
 7,
 7,
 7,
 2,
 6,
 6,
 8,
 4,
 6,
 6,
 0,
 6,
 0,
 6,
 7,
 6,
 8,
 6,
 8,
 1,
 1,
 9,
 2,
 6,
 9,
 6,
 6,
 9,
 7,
 1,
 7,
 5,
 0,
 3,
 3,
 6,
 2,
 1,
 1,
 3,
 2,
 8,
 0,
 2,
 6,
 5,
 1,
 0,
 6,
 0,
 5,
 2,
 2,
 3,
 9,
 5,
 7,
 8,
 2,
 8,
 9,
 7,
 9,
 8,
 7,
 2,
 0,
 8,
 1,
 6,
 1,
 5,
 6,
 5,
 6,
 8,
 2,
 1,
 9,
 2,
 9,
 6,
 5,
 3,
 3,
 5,
 9,
 1,
 2,
 6,
 3,
 5,
 8,
 8,
 7,
 8,
 2,
 3,
 3,
 8,
 6,
 1,
 6,
 8,
 8,
 5,
 6,
 8,
 9,
 9,
 4,
 1,
 8,
 8,
 6,
 0,
 4,
 3,
 5,
 5,
 3,
 7,
 7,
 6,
 1,
 6,
 2,
 7,
 1,
 9,
 2,
 1,
 8,
 1,
 4,
 6,
 5,
 6,
 6,
 7,
 2,
 3,
 0,
 1,
 5,
 6,
 4,
 2,
 6,
 2,
 1,
 7,
 1,
 9,
 9,
 6,
 6,
 8,
 6,
 6,
 7,
 7,
 5,
 9,
 5,
 5,
 6,
 2,
 3,
 2,
 5,
 5,
 6,
 6,
 4,
 9,
 2,
 1,
 0,
 5,
 2,
 6,
 4,
 5,
 0,
 9,
 6,
 6,
 0,
 6,
 6,
 1,
 8,
 1,
 4,
 2,
 1,
 5,
 2,
 0,
 8,
 9,
 6,
 8,
 9,
 3,
 6,
 3,
 9,
 5,


In [53]:
for i in range(y_pred.shape[0]):
    myindex = np.argmax(y_pred[i])
    y_pred[i][myindex] = 1
    y_pred[i][np.where(y_pred[i] != 1)] = 0

In [58]:
for i in range(y_pred.shape[0]):
    y_pred[i] = np.where(y_pred[i] == 1)

In [60]:
y_pred[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
y_pred.shape

(10000, 10)

In [23]:
x_test.shape

(10000, 784)

In [24]:
x_train.shape

(54000, 784)

In [21]:
y_valid

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [22]:
y_valid.shape

(6000, 10)

In [ ]:
# chapter 2 にCSVファイルを提出
# chapter 2 codeにコードを提出

In [ ]:
# 重み (入力の次元数: 784, 出力の次元数: 10)
W_mnist = np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float32')
b_mnist = np.zeros(shape=(10,)).astype('float32')

In [2]:
np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float32')

array([[ 9.49454401e-03, -4.94524948e-02, -1.78046692e-02, ...,
        -2.94937603e-02, -5.65887801e-02,  7.34340921e-02],
       [-7.85746053e-02,  7.04474300e-02,  1.76886078e-02, ...,
        -6.27270192e-02, -3.73890176e-02,  8.63453839e-03],
       [ 3.06786783e-02,  4.98031341e-02,  1.19034285e-02, ...,
         3.53193022e-02, -6.89059380e-05,  3.71921994e-02],
       ...,
       [ 3.16996910e-02,  2.93891933e-02,  5.70208952e-03, ...,
        -2.38972623e-02, -2.21209750e-02, -2.68498212e-02],
       [-7.49154389e-02,  2.58445218e-02,  3.04616261e-02, ...,
         2.02503870e-03, -2.79621128e-02, -2.93700658e-02],
       [ 4.94951233e-02,  3.89414579e-02, -1.65539943e-02, ...,
        -3.45581211e-02, -1.94040854e-02,  4.50179800e-02]], dtype=float32)

In [10]:
np.zeros(shape=(10,)).astype('float32')

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)